<a href="https://colab.research.google.com/github/shikha-aggarwal/researchpaperlikes/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install datasets transformers pytorch-lightning

In [3]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

import transformers

from datasets import (
    list_datasets,
    load_dataset,
    list_metrics,
    load_metric)

import json
import os
import re
import argparse
import random
import time
import string
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler

import matplotlib.pyplot as plt

## Colab accessing Google drive
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Colab\ Notebooks/research\ paper

/content/drive/MyDrive/Colab Notebooks/research paper


In [35]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

In [ ]:
from datasets import load_dataset
from ml_project.data_loaders import load_data

In [17]:
data_dir = './data'
load_tags_list = load_data.load_tags(data_dir)
article_tags_2d_list = load_data.load_article_tags(data_dir)
# citations_2d_np = load_data.load_citations(data_dir, num_articles = )
user_items_2d_np = load_data.load_user_article_likes(data_dir)
articles_df, user_article_likes_2d_np = load_data.load_articles_and_user_article_likes(data_dir)

In [18]:
articles_df.columns

Index(['doc.id', 'title', 'citeulike.id', 'raw.title', 'raw.abstract'], dtype='object')

In [19]:
articles_df.rename(columns = {'citeulike.id':'citeulike_id', 'doc.id': 'doc_id', 'raw.title': 'raw_title', 'raw.abstract': 'raw_abstract'}, inplace = True)

In [20]:
articles_df.head()

,doc_id,title,citeulike_id,raw_title,raw_abstract
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,..."
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...


In [25]:
articles_df.iloc[10].to_dict()

{'citeulike_id': 61.0,
 'doc_id': 11,
 'raw_abstract': ' Recent research has revealed general principles in the structural and functional organization of complex networks which are shared by various natural, social and technological systems. This review examines these principles as applied to the organization, development and function of complex brain networks. Specifically, we examine the structural properties of large-scale anatomical and functional brain networks and discuss how they might arise in the course of network growth and rewiring. Moreover, we examine the relationship between the structural substrate of neuroanatomy and more dynamic functional and effective connectivity patterns that underlie human cognition. We suggest that network analysis offers new fundamental insights into global and integrative aspects of brain function, including the origin of flexible and coherent cognitive states within the neural architecture.',
 'raw_title': 'Organization, development and functi

In [24]:
type(articles_df.iloc[10])

pandas.core.series.Series

In [27]:
class ArticlesDataset_basic(Dataset):

    def __init__(self, pandas_df):
        self.pandas_df = pandas_df

    def __len__(self):
        return len(self.pandas_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.pandas_df.iloc[idx].to_dict()

In [33]:
train_df, validate_df, test_df = np.split(articles_df.sample(frac=1, random_state=42), 
                       [int(.6*len(articles_df)), int(.8*len(articles_df))])

In [45]:
class ArticlesDataset(Dataset):
    def __init__(self, df, tokenizer, input_length):
        self.pandas_df = df
        self.tokenizer = tokenizer         
        self.input_length = input_length
  
    def __len__(self):
        return len(self.pandas_df)
    
    def clean_text(self, text):
        text = text.replace('\n','')
        text = text.replace('``', '')
        text = text.replace('"', '')
        
        return text
    
    def convert_to_features(self, example_batch):
        sentence_1 = self.clean_text(example_batch['raw_title'])  
        sentence_2 = self.clean_text(example_batch['raw_title'])  
        
        source = self.tokenizer.batch_encode_plus([sentence_1], max_length=self.input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
        
        targets = self.tokenizer.batch_encode_plus([sentence_2], max_length=self.input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
    
        return source, targets
  
    def __getitem__(self, index):
        source, targets = self.convert_to_features(self.pandas_df.iloc[index])
        
        source_ids = source["input_ids"].squeeze()
        target_ids = targets["input_ids"].squeeze()

        src_mask    = source["attention_mask"].squeeze()
        target_mask = targets["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

In [36]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
input_length = 100


In [46]:
articles_dataset = ArticlesDataset(train_df, tokenizer = tokenizer, input_length = input_length)

In [ ]:
show_random_elements(articles_dataset)